In [1]:
import os
import sys
import json
root_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(root_path)
import logging
from typing import List, Tuple, Dict

ROOT_DIR = r"C:\Projects\Research\SWEEP\SWEEP"

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def get_sectors() -> List[str]:
    data_path = os.path.join(ROOT_DIR, "data")
    return [sector for sector in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, sector))]

def get_activities(sector: str) -> List[str]:
    sector_path = os.path.join(ROOT_DIR, "data", sector)
    return [activity for activity in os.listdir(sector_path) if os.path.isdir(os.path.join(sector_path, activity))]

def get_file_paths(sector: str, activity: str, model_name: str) -> Tuple[str, str, str, str]:
    dir_path = os.path.join(ROOT_DIR, "data", sector, activity)
    test_path = os.path.join(ROOT_DIR, "test", "results", "activity_breakdown", sector, activity)
    bpmn_model_path = os.path.join(ROOT_DIR, dir_path, f"{activity}_model.txt")
    activity_breakdown_gt = os.path.join(ROOT_DIR, dir_path, f"{activity}_activity_breakdown.json")
    response_path = os.path.join(ROOT_DIR, test_path, f"{model_name}_response.json")
    return test_path, bpmn_model_path, activity_breakdown_gt, response_path

In [2]:
from src.common.prompts.activity_breakdown.prompt_components import ABPromptComponents
from src.activity_breakdown import ActivityBreakdownComponentsGPT, ActivityBreakdownEvaluation
from src.common.models import ActivityBreakdown, BPMNModel
from src.common.util import open_file

activity_breakdown_evaluation: ActivityBreakdownEvaluation = ActivityBreakdownEvaluation()

def breakdown_activity(sector: str, activity: str, activity_breakdown_gpt: ActivityBreakdownComponentsGPT, model_name: str) -> None:
    logging.info(f"Processing sector: {sector}, activity: {activity}")
    
    try:
        # Load activity breakdown and ground truth
        test_path, bpmn_model_path, activity_breakdown_gt, response_path = get_file_paths(sector, activity, model_name)
        
        os.makedirs(test_path, exist_ok=True)
        
        bpmn_model = BPMNModel.from_text(bpmn_model_path)
        ground_truth: ActivityBreakdown = ActivityBreakdown.from_json(activity_breakdown_gt)
        
        activity_breakdown_response = activity_breakdown_gpt.bpmn_activity_breakdown(bpmn_model.model)
        model_activity_breakdown: ActivityBreakdown = ActivityBreakdown.from_dict(activity, activity_breakdown_response)
        
        # Compare results
        comparison_data = activity_breakdown_evaluation.compare_activity_breakdowns(model_activity_breakdown.to_dict(), ground_truth.to_dict())
        comparison_metrics = activity_breakdown_evaluation.evaluate_comparisons(comparison_data, plot=False)
        
        response_dict = {
            "model": {
                "name": model_name,
                "components": activity_breakdown_gpt.prompt_components["_raw_input"]
            },
            "response": model_activity_breakdown.to_dict(),
            "metrics": comparison_metrics
        }
        
        # Save results
        with open(response_path, 'w') as f:
            json.dump(response_dict, f, indent=4)
        
        logging.info(f"Successfully processed and saved results for {sector}/{activity} with {model_name}")
    
    except Exception as e:
        logging.error(f"Error processing {sector}/{activity}: {str(e)}")
        

In [3]:
model_name = "GPT-3.5-Activity-Breakdown-Subject_Matter_Expert_Detailed_Basic"

model_settings = {
        'role_description': "sme_detailed",
        'task_description': "breakdown_substeps",
        'guidelines': "standard_guidelines",
        'output_format': "standard_output",
        # 'example_output': "",
        # 'focus_shift': "",
}

model_components = ABPromptComponents.from_dict(model_settings)


activity_breakdown_gpt = ActivityBreakdownComponentsGPT(model_components)

In [4]:
def main():
    logging.info(f"Processing with model: {model_name}")
    
    sectors = get_sectors()
    for sector in sectors:
        if "sme" in model_settings.get("role_description",""):
            components = ABPromptComponents.from_dict(model_settings)
            components.role_description = components.role_description[sector]
            activity_breakdown_gpt = ActivityBreakdownComponentsGPT(components)
        activities = get_activities(sector)
        for activity in activities:
            
            breakdown_activity(sector, activity, activity_breakdown_gpt, model_name)

if __name__ == "__main__":
    main()

2024-09-26 14:22:20,072 - INFO - Processing with model: GPT-3.5-Activity-Breakdown-Subject_Matter_Expert_Detailed_Basic
2024-09-26 14:22:20,312 - INFO - Processing sector: banking, activity: credit_application
2024-09-26 14:22:26,874 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"


Comparing activity 1...


2024-09-26 14:22:30,444 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"


Comparing activity 2...


2024-09-26 14:22:34,051 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"


...

Comparing activity 8...


2024-09-26 14:46:00,136 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"


Comparing activity 9...


2024-09-26 14:46:02,878 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-09-26 14:46:02,882 - INFO - Successfully processed and saved results for insurance/quote_submission with GPT-3.5-Activity-Breakdown-Subject_Matter_Expert_Detailed_Basic


Quantitative Metrics:
Total Activities Evaluated: 4
Average Overall Similarity Score: 59.07%
Average Confidence Score: 55.71%
Average Sequence Alignment Score: 69.44%
Ground Truth Steps Coverage: 100.00%
Response Steps Coverage: 62.16%

Match Type Distribution:
- Granularity Difference: 3 matches (21.43%), Average Confidence: 76.67%
- Functional Equivalence: 6 matches (42.86%), Average Confidence: 80.00%
- Process Alternative: 1 matches (7.14%), Average Confidence: 70.00%
- No Match: 4 matches (28.57%), Average Confidence: 0.00%


Qualitative Analysis:

Examples of Matched Steps by Match Type:

Match Type: Granularity Difference
Activity: Request insurance quote
Ground Truth Steps:
  - [1] Client requests insurance quote
Response Steps:
  - [1] Identify insurance needs
  - [2] Gather necessary personal and property information
  - [3] Contact insurer or fill out online quote request form
  - [4] Submit quote request
Confidence: 80%
Explanation: The ground truth step 'Client requests in